In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, scorer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import LocalOutlierFactor
import joblib
from joblib import dump, load
import hypopt
from hypopt import GridSearch
import pickle
from tensorflow import keras
import kerastuner as kt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.constraints import max_norm
from keras import backend as K
import random

In [2]:
X_all = pd.read_pickle("X_all.pkl")
Y_all = pd.read_pickle("Y_all.pkl")

In [ ]:
estimator.fit(X_train)

In [11]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
final_cm=np.array([[0, 0],[0, 0]])

dataset_covid_healthy = pd.read_pickle("dataset_covid_healthy.pkl")
uuids_status_unique= dataset_covid_healthy.filter(items=['uuid','status']).drop_duplicates()
uuids = np.array(uuids_status_unique['uuid'])
y_uuids = np.array(uuids_status_unique['status'])


for index, (train, test) in enumerate(skf.split(uuids, y_uuids)):
    print('Iteration ', index)
    train_val = set(train)
    val= list(random.sample(train_val, 234))
    train = list(train_val - set(val))

    train_uuids = uuids[train]
    val_uuids = uuids[val]
    test_uuids = uuids[test]
    
    # Creating folds and validation set for tuning
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    
    for x in dataset_covid_healthy.iterrows():
        if x[1]['uuid'] in train_uuids:
            if x[1]['status']=='healthy':
                X_train.append(X_all.loc[x[0]])
                y_train.append(-1)
        elif x[1]['uuid'] in test_uuids:
            X_test.append(X_all.loc[x[0]])
            y_test.append(1 if x[1]['status']=='healthy' else -1)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    estimator = LocalOutlierFactor(novelty=True)
    estimator.fit(X_train, y_train)
    
    preds = estimator.predict(X_test)
    final_cm += confusion_matrix(y_test, preds)
    
    print(confusion_matrix(y_test, preds))
    print('\n')

Iteration  0
[[ 13 213]
 [  6 206]]


Iteration  1
[[  3 179]
 [ 13 194]]


Iteration  2
[[  7 162]
 [ 11 196]]


Iteration  3
[[  7 167]
 [  6 196]]


Iteration  4
[[  8 220]
 [ 17 220]]


Iteration  5
[[ 11 155]
 [ 14 172]]


Iteration  6
[[  4 181]
 [  6 236]]


Iteration  7
[[  8 147]
 [  0 222]]


Iteration  8
[[ 13 155]
 [ 15 161]]


Iteration  9
[[ 11 165]
 [  6 191]]


